In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [ ]:

# Image dimensions
img_height, img_width = 64, 64

# Path to the dataset directory
dataset_dir = 'Dataset'

# Data augmentation and preprocessing with training-validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Increased rotation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.7, 1.3],  # Adjust contrast/brightness
    horizontal_flip=False,
    validation_split=0.2
)



In [ ]:
# Load training data (80% of the dataset)
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',  # Load grayscale images
    subset='training'  # Use the 'training' subset
)

# Load validation data (20% of the dataset)
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',  # Load grayscale images
    subset='validation'  # Use the 'validation' subset
)


In [ ]:
import matplotlib.pyplot as plt

# Get a batch of images and labels from the training generator
images, labels = next(train_generator)

# Set up the figure
plt.figure(figsize=(10, 10))

# Plot the first 9 images
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.title(f'Label: {labels[i].argmax()}')  # Show the index of the class
    plt.axis('off')

plt.show()


In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 1)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(26, activation='softmax')  # 26 classes for letters
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Compile the model

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[early_stopping])


In [ ]:
# Save the model if needed
model.save('Model_Weights/letter_recognition_cnn_st.h5')

In [ ]:
# loading the model if needed
from tensorflow.keras.models import load_model
model = load_model('Model_Weights/letter_recognition_cnn_st.h5')


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Testing the model 


# Load and preprocess the image
img_path = '696969.jpg'
img = image.load_img(img_path, target_size=(64, 64), color_mode='grayscale')
img_array = image.img_to_array(img)

# Data augmentation-like preprocessing (just rescaling here)
img_array /= 255.0

# Add batch dimension
img_array = np.expand_dims(img_array, axis=0)

# Make predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

# Class labels
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
print(f'Predicted class: {class_labels[predicted_class]}')




# Hesitation is Defeat